In [1]:
import pandas as pd
import os
import shutil
from PIL import Image

In [2]:
df = pd.read_csv('CBIS-DDSM_combined.csv')
df.head()

,pathology,left or right breast,image view,abnormality type,image_path
0,MALIGNANT,LEFT,CC,mass,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.342386...
1,MALIGNANT,LEFT,MLO,mass,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.359308...
2,BENIGN,LEFT,CC,mass,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.891800...
3,BENIGN,LEFT,MLO,mass,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.295360...
4,BENIGN,RIGHT,MLO,mass,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.410524...


In [6]:
# Define the folder paths
base_folder = 'Temp'  # Define where you want to save images
folders = {
    ('LEFT', 'CC'): os.path.join(base_folder, 'left_cc'),
    ('LEFT', 'MLO'): os.path.join(base_folder, 'left_mlo'),
    ('RIGHT', 'CC'): os.path.join(base_folder, 'right_cc'),
    ('RIGHT', 'MLO'): os.path.join(base_folder, 'right_mlo')
}

In [7]:
# Create directories if they do not exist
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

In [8]:
# Function to copy files to corresponding folders
def copy_images(row):
    breast_side = row['left or right breast']
    image_view = row['image view']
    image_path = row['image_path']
    
    # Find destination folder
    dest_folder = folders.get((breast_side, image_view))
    
    if dest_folder:
        # Get the filename
        image_name = os.path.basename(image_path)
        # Define the destination path
        dest_path = os.path.join(dest_folder, image_name)
        # Copy the image to the destination folder
        shutil.copy(image_path, dest_path)

In [9]:
# Apply the function to each row in the dataset
df.apply(copy_images, axis=1)

print("Images have been copied successfully!")

Images have been copied successfully!


## based on B/M Classes

In [7]:
# Define the base folder where the images will be copied
base_folder = 'Temp'  # Define where you want to save images

# Define the folder structure for benign and malignant
folders = {
    ('BENIGN', 'LEFT', 'CC'): os.path.join(base_folder, 'benign', 'left_cc'),
    ('BENIGN', 'LEFT', 'MLO'): os.path.join(base_folder, 'benign', 'left_mlo'),
    ('BENIGN', 'RIGHT', 'CC'): os.path.join(base_folder, 'benign', 'right_cc'),
    ('BENIGN', 'RIGHT', 'MLO'): os.path.join(base_folder, 'benign', 'right_mlo'),
    ('MALIGNANT', 'LEFT', 'CC'): os.path.join(base_folder, 'malignant', 'left_cc'),
    ('MALIGNANT', 'LEFT', 'MLO'): os.path.join(base_folder, 'malignant', 'left_mlo'),
    ('MALIGNANT', 'RIGHT', 'CC'): os.path.join(base_folder, 'malignant', 'right_cc'),
    ('MALIGNANT', 'RIGHT', 'MLO'): os.path.join(base_folder, 'malignant', 'right_mlo')
}

# Create directories if they do not exist
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

In [8]:
# Function to copy files to corresponding folders
def copy_images(row):
    pathology = row['pathology'].upper()  # BENIGN or MALIGNANT
    breast_side = row['left or right breast']
    image_view = row['image view']
    image_path = row['image_path']
    
    # Find destination folder
    dest_folder = folders.get((pathology, breast_side, image_view))
    
    if dest_folder:
        # Get the filename
        image_name = os.path.basename(image_path)
        # Define the destination path
        dest_path = os.path.join(dest_folder, image_name)
        # Copy the image to the destination folder
        shutil.copy(image_path, dest_path)

In [9]:
# Apply the function to each row in the dataset
df.apply(copy_images, axis=1)
print("Images have been copied successfully!")

Images have been copied successfully!


## Code to align the Images

In [10]:
# Define the base folder where the images will be copied
base_folder = 'Aligned'  # Define where you want to save images

# Define the folder structure for benign and malignant
folders = {
    ('BENIGN', 'LEFT', 'CC'): os.path.join(base_folder, 'benign', 'left_cc'),
    ('BENIGN', 'LEFT', 'MLO'): os.path.join(base_folder, 'benign', 'left_mlo'),
    ('BENIGN', 'RIGHT', 'CC'): os.path.join(base_folder, 'benign', 'right_cc'),
    ('BENIGN', 'RIGHT', 'MLO'): os.path.join(base_folder, 'benign', 'right_mlo'),
    ('MALIGNANT', 'LEFT', 'CC'): os.path.join(base_folder, 'malignant', 'left_cc'),
    ('MALIGNANT', 'LEFT', 'MLO'): os.path.join(base_folder, 'malignant', 'left_mlo'),
    ('MALIGNANT', 'RIGHT', 'CC'): os.path.join(base_folder, 'malignant', 'right_cc'),
    ('MALIGNANT', 'RIGHT', 'MLO'): os.path.join(base_folder, 'malignant', 'right_mlo')
}

# Create directories if they do not exist
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

## Code for Left Breast

In [11]:
import os
import cv2
import numpy as np

### LEFT CC -> BENIGN
# Define the input and output folders
input_folder = 'Temp/benign/left_cc'  # Folder containing the original images
output_folder = 'Aligned/benign/left_cc'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the right
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the left half has more intensity, assume it's aligned to the left and flip it
    if left_sum > right_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the right if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")


Aligned and saved: 1-003.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-005.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-011.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-015.jpg
Aligned and saved: 1-016.jpg
Aligned and saved: 1-017.jpg
Aligned and saved: 1-018.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-029.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-032.jpg
Aligned and saved: 1-033.jpg
Aligned and saved: 1-034.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-038.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-040.jpg
Aligned and saved: 1-041.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-047.jpg
Aligned and saved: 1-048.jpg
Aligned and saved: 1-050.jpg
Aligned and sa

In [12]:
import os
import cv2
import numpy as np

### LEFT MLO -> BENIGN
# Define the input and output folders
input_folder = 'Temp/benign/left_mlo'  # Folder containing the original images
output_folder = 'Aligned/benign/left_mlo'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the right
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the left half has more intensity, assume it's aligned to the left and flip it
    if left_sum > right_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the right if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")


Aligned and saved: 1-000.jpg
Aligned and saved: 1-001.jpg
Aligned and saved: 1-003.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-005.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-007.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-009.jpg
Aligned and saved: 1-012.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-015.jpg
Aligned and saved: 1-016.jpg
Aligned and saved: 1-017.jpg
Aligned and saved: 1-018.jpg
Aligned and saved: 1-019.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-022.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-029.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-031.jpg
Aligned and saved: 1-033.jpg
Aligned and saved: 1-034.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-041.jpg
Aligned and saved: 1-042.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-044.jpg
Aligned and sa

In [13]:
import os
import cv2
import numpy as np

### LEFT CC -> MALIGNANT
# Define the input and output folders
input_folder = 'Temp/malignant/left_cc'  # Folder containing the original images
output_folder = 'Aligned/malignant/left_cc'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the right
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the left half has more intensity, assume it's aligned to the left and flip it
    if left_sum > right_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the right if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")


Aligned and saved: 1-001.jpg
Aligned and saved: 1-002.jpg
Aligned and saved: 1-003.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-005.jpg
Aligned and saved: 1-007.jpg
Aligned and saved: 1-010.jpg
Aligned and saved: 1-011.jpg
Aligned and saved: 1-016.jpg
Aligned and saved: 1-018.jpg
Aligned and saved: 1-019.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-022.jpg
Aligned and saved: 1-023.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-029.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-031.jpg
Aligned and saved: 1-032.jpg
Aligned and saved: 1-033.jpg
Aligned and saved: 1-034.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-038.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-041.jpg
Aligned and saved: 1-042.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-044.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-048.jpg
Aligned and sa

In [14]:
import os
import cv2
import numpy as np

### LEFT MLO -> MALIGNANT
# Define the input and output folders
input_folder = 'Temp/malignant/left_mlo'  # Folder containing the original images
output_folder = 'Aligned/malignant/left_mlo'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the right
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the left half has more intensity, assume it's aligned to the left and flip it
    if left_sum > right_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the right if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")


Aligned and saved: 1-000.jpg
Aligned and saved: 1-002.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-009.jpg
Aligned and saved: 1-011.jpg
Aligned and saved: 1-012.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-015.jpg
Aligned and saved: 1-017.jpg
Aligned and saved: 1-019.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-022.jpg
Aligned and saved: 1-023.jpg
Aligned and saved: 1-024.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-029.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-031.jpg
Aligned and saved: 1-032.jpg
Aligned and saved: 1-034.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-038.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-040.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-044.jpg
Aligned and sa

## Code for Right Breast

In [15]:
import os
import cv2
import numpy as np

#### RIGHT CC -> BENIGN
# Define the input and output folders
input_folder = 'Temp/benign/right_cc'  # Folder containing the original images
output_folder = 'Aligned/benign/right_cc'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the left
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the right half has more intensity, assume it's aligned to the right and flip it
    if right_sum > left_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the left if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")

Aligned and saved: 1-000.jpg
Aligned and saved: 1-002.jpg
Aligned and saved: 1-003.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-007.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-009.jpg
Aligned and saved: 1-010.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-016.jpg
Aligned and saved: 1-018.jpg
Aligned and saved: 1-019.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-022.jpg
Aligned and saved: 1-024.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-031.jpg
Aligned and saved: 1-034.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-038.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-040.jpg
Aligned and saved: 1-042.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-046.jpg
Aligned and saved: 1-049.jpg
Aligned and saved: 1-050.jpg
Aligned and saved: 1-052.jpg
Aligned and sa

In [16]:
import os
import cv2
import numpy as np

#### RIGHT MLO -> BENIGN
# Define the input and output folders
input_folder = 'Temp/benign/right_mlo'  # Folder containing the original images
output_folder = 'Aligned/benign/right_mlo'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the left
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the right half has more intensity, assume it's aligned to the right and flip it
    if right_sum > left_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the left if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")

Aligned and saved: 1-001.jpg
Aligned and saved: 1-002.jpg
Aligned and saved: 1-003.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-007.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-009.jpg
Aligned and saved: 1-010.jpg
Aligned and saved: 1-012.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-015.jpg
Aligned and saved: 1-017.jpg
Aligned and saved: 1-019.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-040.jpg
Aligned and saved: 1-041.jpg
Aligned and saved: 1-042.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-044.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-046.jpg
Aligned and saved: 1-047.jpg
Aligned and saved: 1-048.jpg
Aligned and saved: 1-050.jpg
Aligned and saved: 1-051.jpg
Aligned and sa

In [17]:
import os
import cv2
import numpy as np

#### RIGHT CC -> MALIGNANT
# Define the input and output folders
input_folder = 'Temp/malignant/right_cc'  # Folder containing the original images
output_folder = 'Aligned/malignant/right_cc'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the left
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the right half has more intensity, assume it's aligned to the right and flip it
    if right_sum > left_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the left if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")

Aligned and saved: 1-001.jpg
Aligned and saved: 1-002.jpg
Aligned and saved: 1-003.jpg
Aligned and saved: 1-004.jpg
Aligned and saved: 1-008.jpg
Aligned and saved: 1-011.jpg
Aligned and saved: 1-012.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-020.jpg
Aligned and saved: 1-021.jpg
Aligned and saved: 1-023.jpg
Aligned and saved: 1-024.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-028.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-032.jpg
Aligned and saved: 1-033.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-041.jpg
Aligned and saved: 1-043.jpg
Aligned and saved: 1-044.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-048.jpg
Aligned and saved: 1-049.jpg
Aligned and saved: 1-050.jpg
Aligned and saved: 1-052.jpg
Aligned and saved: 1-055.jpg
Aligned and saved: 1-057.jpg
Aligned and saved: 1-059.jpg
Aligned and saved: 1-060.jpg
Aligned and saved: 1-062.jpg
Aligned and saved: 1-064.jpg
Aligned and sa

In [18]:
import os
import cv2
import numpy as np

#### RIGHT MLO -> MALIGNANT
# Define the input and output folders
input_folder = 'Temp/malignant/right_mlo'  # Folder containing the original images
output_folder = 'Aligned/malignant/right_mlo'  # Folder to save aligned images
os.makedirs(output_folder, exist_ok=True)

# Function to check image alignment and align to the left
def align_image(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the width of the image
    height, width = gray_image.shape
    
    # Sum the pixel intensities along the vertical axis (columns)
    left_sum = np.sum(gray_image[:, :width // 2])  # Left half
    right_sum = np.sum(gray_image[:, width // 2:])  # Right half
    
    # If the right half has more intensity, assume it's aligned to the right and flip it
    if right_sum > left_sum:
        image = cv2.flip(image, 1)  # Flip horizontally (left-right)
    
    return image

# Process each image in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    image = cv2.imread(image_path)
    
    if image is not None:
        # Align the image to the left if necessary
        aligned_image = align_image(image)
        
        # Save the aligned image to the output folder
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, aligned_image)
        print(f"Aligned and saved: {image_name}")
    else:
        print(f"Failed to load image: {image_name}")

print("All images have been processed and aligned!")

Aligned and saved: 1-004.jpg
Aligned and saved: 1-005.jpg
Aligned and saved: 1-006.jpg
Aligned and saved: 1-009.jpg
Aligned and saved: 1-010.jpg
Aligned and saved: 1-011.jpg
Aligned and saved: 1-012.jpg
Aligned and saved: 1-013.jpg
Aligned and saved: 1-014.jpg
Aligned and saved: 1-015.jpg
Aligned and saved: 1-016.jpg
Aligned and saved: 1-017.jpg
Aligned and saved: 1-018.jpg
Aligned and saved: 1-022.jpg
Aligned and saved: 1-023.jpg
Aligned and saved: 1-025.jpg
Aligned and saved: 1-026.jpg
Aligned and saved: 1-027.jpg
Aligned and saved: 1-030.jpg
Aligned and saved: 1-031.jpg
Aligned and saved: 1-033.jpg
Aligned and saved: 1-035.jpg
Aligned and saved: 1-036.jpg
Aligned and saved: 1-037.jpg
Aligned and saved: 1-038.jpg
Aligned and saved: 1-039.jpg
Aligned and saved: 1-040.jpg
Aligned and saved: 1-042.jpg
Aligned and saved: 1-045.jpg
Aligned and saved: 1-046.jpg
Aligned and saved: 1-047.jpg
Aligned and saved: 1-048.jpg
Aligned and saved: 1-051.jpg
Aligned and saved: 1-052.jpg
Aligned and sa